<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/Train_Accidents_Pred_road.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd

#【祝日】
holi_df = pd.read_csv("./drive/My Drive/TrainAccident/holiday.csv")
holi_df["date"] = holi_df['date'].astype('str')

#【土日】
donichi_df = pd.read_csv("./drive/My Drive/TrainAccident/donichi.csv",index_col=0)
donichi_df["date"] = donichi_df['date'].astype('str')
donichi_list = donichi_df['date'].values.tolist()

#【休み明け】
renkyu_ake_df = pd.read_csv("./drive/My Drive/TrainAccident/renkyu_ake.csv")
renkyu_ake_df["date"] = renkyu_ake_df['date'].astype('str')
renkyu_ake_list = renkyu_ake_df['date'].values.tolist()

#【気温】
temp_df = pd.read_csv("./drive/My Drive/TrainAccident/temp.csv")
temp_df = temp_df.dropna(subset=["max_temp"])
temp_df = temp_df.dropna(subset=["min_temp"])
temp_df["date"] = temp_df['date'].astype('str')

#項目追加
temp_df = temp_df.assign(diff_temp=temp_df['max_temp'].astype(float)-temp_df['min_temp'].astype(float))
temp_df = temp_df.assign(max_temp_std=(temp_df["max_temp"] - temp_df["max_temp"].min()) / (temp_df["max_temp"].max() - temp_df["max_temp"].min()))
temp_df = temp_df.assign(min_temp_std=(temp_df["min_temp"] - temp_df["min_temp"].min()) / (temp_df["min_temp"].max() - temp_df["min_temp"].min()))
temp_df = temp_df.assign(diff_temp_std=(temp_df["diff_temp"] - temp_df["diff_temp"].min()) / (temp_df["diff_temp"].max() - temp_df["diff_temp"].min()))

#【日照時間】
sun_df = pd.read_csv("./drive/My Drive/TrainAccident/suntime.csv")
sun_df = sun_df.dropna(subset=["suntime"])
sun_df["date"] = sun_df['date'].astype('str')

#項目追加
sun_df = sun_df.assign(suntime_std=(sun_df["suntime"] - sun_df["suntime"].min()) / (sun_df["suntime"].max() - sun_df["suntime"].min()))

#【降水量/気圧】
rain_pres_df = pd.read_csv("./drive/My Drive/TrainAccident/rain_pres.csv")
rain_pres_df = rain_pres_df.drop('dmy',axis=1)
rain_pres_df["date"] = rain_pres_df['date'].astype('str')

#項目追加
rain_pres_df = rain_pres_df.assign(rain_std=(rain_pres_df["rain"] - rain_pres_df["rain"].min()) / (rain_pres_df["rain"].max() - rain_pres_df["rain"].min()))
rain_pres_df = rain_pres_df.assign(pres_std=(rain_pres_df["pres"] - rain_pres_df["pres"].min()) / (rain_pres_df["pres"].max() - rain_pres_df["pres"].min()))

In [56]:
import numpy as np
import pandas as pd
from datetime import date, timedelta

v_date = "2019-08-26"

#日付データ作成
dates_df = pd.DataFrame({"date": [pd.to_datetime(v_date) + timedelta(days=i) for i in range(1)]},dtype="object")
date_list = dates_df.values.tolist()

#デフォルトでは小数点が付いてしまう為、int型に変換
time_cate = np.eye(7).astype(int)
yobi_cate = np.eye(7).astype(int)
#holiday関連(休日flg,休日明けflg)
#ninzu_cate = np.eye(5).astype(int)

wk_list = []

for wk in date_list:
  for wk2 in time_cate:
    for wk3 in yobi_cate:
      #人数(正規化済)分 繰り返し
      list_parts = np.concatenate((wk,wk2,wk3,0,0,0.00), axis=None)
      wk_list.append(list_parts.tolist())
      list_parts = np.concatenate((wk,wk2,wk3,0,0,0.01), axis=None)
      wk_list.append(list_parts.tolist())
      list_parts = np.concatenate((wk,wk2,wk3,0,0,0.02), axis=None)
      wk_list.append(list_parts.tolist())
      list_parts = np.concatenate((wk,wk2,wk3,0,0,0.04), axis=None)
      wk_list.append(list_parts.tolist())
      list_parts = np.concatenate((wk,wk2,wk3,0,0,0.08), axis=None)
      wk_list.append(list_parts.tolist())
      list_parts = np.concatenate((wk,wk2,wk3,0,0,0.16), axis=None)
      wk_list.append(list_parts.tolist())
      list_parts = np.concatenate((wk,wk2,wk3,0,0,0.32), axis=None)
      wk_list.append(list_parts.tolist())
      list_parts = np.concatenate((wk,wk2,wk3,0,0,0.64), axis=None)
      wk_list.append(list_parts.tolist())
      list_parts = np.concatenate((wk,wk2,wk3,0,0,1.00), axis=None)
      wk_list.append(list_parts.tolist())
          
col=["date",
     "comp_time","day_am_time","day_pm_time","home_time","midnight_time","morning_time","night_time",
     "yobi_0","yobi_1","yobi_2","yobi_3","yobi_4","yobi_5","yobi_6",
     "holiday_flg","holi_ake_flg",
     "ninzu"]

test_df = pd.DataFrame(data=wk_list,columns=col)

test_df['date'] = test_df['date'].astype('str')
test_df['date'] = test_df['date'].str.replace('-', '/')

#土日
test_df.loc[(test_df['date'].str[5:10].isin(donichi_list)) ,"holiday_flg"] = 1

#祝日
Special_holiday_list = ["08/13","08/14","08/15","08/16","12/29","12/30","12/31","01/02","01/03","01/04"]
test_df.loc[(test_df['date'].str[5:10].isin(Special_holiday_list)) ,"holiday_flg"] = 1
test_df.loc[(test_df['holiday_flg'].isnull()) ,"holiday_flg"] = 0

#連休明け
ake_list = renkyu_ake_df["date"].values.tolist()
test_df.loc[(test_df["date"].isin(ake_list)),"holi_ake_flg"] = 1
test_df.loc[(test_df['holi_ake_flg'].isnull()) ,"holi_ake_flg"] = 0

#不要な曜日データは削除
yobi_code = pd.to_datetime(v_date).weekday()
test_df = test_df[test_df["yobi_"+str(yobi_code)] == 1]

#日付削除
test_df2 = test_df.drop(["date"],axis=1)
test_df2.to_csv("./drive/My Drive/TrainAccident/test_in_data.csv")

#気象条件設定
v_max_temp = 35
v_min_temp = 25
v_diff_temp = v_max_temp - v_min_temp
v_sun = 7
v_rain = 0
v_pres = 998

test_df2 = test_df2.assign(max_temp=temp_df[(temp_df["max_temp"] <= v_max_temp + 0.2) & (temp_df["max_temp"] >= v_max_temp - 0.2)]["max_temp_std"].mean())
test_df2 = test_df2.assign(min_temp=temp_df[(temp_df["min_temp"] <= v_min_temp + 0.2) & (temp_df["min_temp"] >= v_min_temp - 0.2)]["min_temp_std"].mean())
test_df2 = test_df2.assign(diff_temp=temp_df[(temp_df["diff_temp"] <= v_diff_temp + 0.2) & (temp_df["diff_temp"] >= v_diff_temp - 0.2)]["diff_temp_std"].mean())
test_df2 = test_df2.assign(suntime_temp=sun_df[(sun_df["suntime"] <= v_sun + 0.5) & (sun_df["suntime"] >= v_sun - 0.5)]["suntime_std"].mean())
test_df2 = test_df2.assign(rain=rain_pres_df[(rain_pres_df["rain"] <= v_rain + 1) & (rain_pres_df["rain"] >= v_rain - 1)]["rain_std"].mean())
test_df2 = test_df2.assign(pres=rain_pres_df[(rain_pres_df["pres"] <= v_pres + 1) & (rain_pres_df["pres"] >= v_pres - 1)]["pres_std"].mean())

#検証用データはreset_indexしないと欠番の行が出来る
test_df2 = test_df2.reset_index(drop=True)
print(test_df2.shape)
test_df2.head(5)


(63, 23)


,comp_time,day_am_time,day_pm_time,home_time,midnight_time,morning_time,night_time,yobi_0,yobi_1,yobi_2,yobi_3,yobi_4,yobi_5,yobi_6,holiday_flg,holi_ake_flg,ninzu,max_temp,min_temp,diff_temp,suntime_temp,rain,pres
0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0.00,0.904991,0.872551,0.483524,0.516063,0.000291,0.383995
1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0.01,0.904991,0.872551,0.483524,0.516063,0.000291,0.383995
2,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0.02,0.904991,0.872551,0.483524,0.516063,0.000291,0.383995
3,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0.04,0.904991,0.872551,0.483524,0.516063,0.000291,0.383995
4,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0.08,0.904991,0.872551,0.483524,0.516063,0.000291,0.383995


In [0]:
#保存したモデルをロードする(RFC)
import pickle
import pandas as pd

filename = './drive/My Drive/TrainAccident/Accident_RFC_model.sav'
loaded_RFC_model = pickle.load(open(filename, 'rb'))

In [59]:
# 評価する
y_pred = loaded_RFC_model.predict(test_df2)

accident_df = pd.DataFrame(y_pred,columns=["Accident"])
result_df = pd.concat([test_df2,accident_df],axis=1)

result_df = result_df.drop(["yobi_0","yobi_1","yobi_2","yobi_3","yobi_4","yobi_5","yobi_6","holiday_flg","holi_ake_flg","max_temp","min_temp","diff_temp","suntime_temp","rain","pres"],axis=1)
#result_df = result_df[result_df["Accident"]==1]
result_df

,comp_time,day_am_time,day_pm_time,home_time,midnight_time,morning_time,night_time,ninzu,Accident
0,1,0,0,0,0,0,0,0.00,0
1,1,0,0,0,0,0,0,0.01,0
2,1,0,0,0,0,0,0,0.02,0
3,1,0,0,0,0,0,0,0.04,0
4,1,0,0,0,0,0,0,0.08,0
5,1,0,0,0,0,0,0,0.16,0
6,1,0,0,0,0,0,0,0.32,0
7,1,0,0,0,0,0,0,0.64,0
8,1,0,0,0,0,0,0,1.00,0
9,0,1,0,0,0,0,0,0.00,0
